# Funkcja klasyfikacji

In [3]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
import numpy as np


def score_dataset(x_train, x_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(x_train, y_train)
    preds = model.predict(x_valid)
    return mean_absolute_error(y_valid, preds)

# Wczytanie danych

In [4]:
import pandas as pd

df = pd.read_csv('melb_data.csv')

# Podział danych

In [5]:
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split


train_df, test_df = train_test_split(df, test_size=0.7)

# Przechowywanie wyników testów

In [6]:
results = {}

# Brakujace kolumny

In [7]:
missing_values_count = df.isnull().sum().loc[lambda x: x > 0]

print(f'Missing columns:\n{missing_values_count}')

missing_values_percentage = (100 * (df.isnull().sum() / len(df))).round(2)

lack = df.isnull().sum().sum()
not_lack = df.notnull().sum().sum()

d = {lack}

total_number_of_null = pd.DataFrame(d, index=['Total number of null values: '])

missing_values_percentage = pd.concat([missing_values_percentage, total_number_of_null], sort=False)

missing_values_percentage.columns = ['Values: ']

print(f'Missing columns:\n{missing_values_percentage}')

Missing columns:
Car               62
BuildingArea    6450
YearBuilt       5375
CouncilArea     1369
dtype: int64
Missing columns:
                               Values: 
Suburb                             0.00
Address                            0.00
Rooms                              0.00
Type                               0.00
Price                              0.00
Method                             0.00
SellerG                            0.00
Date                               0.00
Distance                           0.00
Postcode                           0.00
Bedroom2                           0.00
Bathroom                           0.00
Car                                0.46
Landsize                           0.00
BuildingArea                      47.50
YearBuilt                         39.58
CouncilArea                       10.08
Lattitude                          0.00
Longtitude                         0.00
Regionname                         0.00
Propertycount                

# Usunięcie kolumn zawierających przynajmniej 1 element pusty

In [8]:
train_df_cleaned = train_df.dropna()
test_df_cleaned = test_df.dropna()

cols_x = train_df_cleaned.select_dtypes(include=[np.number]).columns.difference(['Price'])  # wybiera tylko kolumny z wartosciami numerycznymi, za wyjątkiem kolumny z wartością referencyjną - wejście do klasyfikatora
cols_y = 'Price'  # - wyjście z klasyfikatora
score = score_dataset(train_df_cleaned[cols_x], test_df_cleaned[cols_x], train_df_cleaned[cols_y], test_df_cleaned[cols_y])
print(score)

results['Column erased'] = score

183953.80955050973


# Usunięcie wierszy zawierających przynajmniej 1 element pusty

In [9]:
train_df_cleaned = train_df.dropna(axis=1)
test_df_cleaned = test_df.dropna(axis=1)

cols_x = train_df_cleaned.select_dtypes(include=[np.number]).columns.difference(['Price'])  # wybiera tylko kolumny z wartosciami numerycznymi, za wyjątkiem kolumny z wartością referencyjną - wejście do klasyfikatora
cols_y = 'Price'  # - wyjście z klasyfikatora
score = score_dataset(train_df_cleaned[cols_x], test_df_cleaned[cols_x], train_df_cleaned[cols_y], test_df_cleaned[cols_y])
print(score)

results['Rows erased'] = score

192095.01516671173


# Usunięcie kolumn i wierszy zawierających przynajmniej 1 element pusty

In [10]:
train_df_cleaned = train_df.dropna()
test_df_cleaned = test_df.dropna()

train_df_cleaned = train_df_cleaned.dropna(axis=1)
test_df_cleaned = test_df_cleaned.dropna(axis=1)

cols_x = train_df_cleaned.select_dtypes(include=[np.number]).columns.difference(['Price'])  # wybiera tylko kolumny z wartosciami numerycznymi, za wyjątkiem kolumny z wartością referencyjną - wejście do klasyfikatora
cols_y = 'Price'  # - wyjście z klasyfikatora
print(score_dataset(train_df_cleaned[cols_x], test_df_cleaned[cols_x], train_df_cleaned[cols_y], test_df_cleaned[cols_y]))

183953.80955050973


# Wypełnienie pustych wartości zerami

In [11]:
train_df_cleaned = train_df.fillna(0)
test_df_cleaned = test_df.fillna(0)

cols_x = train_df_cleaned.select_dtypes(include=[np.number]).columns.difference(['Price'])  # wybiera tylko kolumny z wartosciami numerycznymi, za wyjątkiem kolumny z wartością referencyjną - wejście do klasyfikatora
cols_y = 'Price'  # - wyjście z klasyfikatora
score = score_dataset(train_df_cleaned[cols_x], test_df_cleaned[cols_x], train_df_cleaned[cols_y], test_df_cleaned[cols_y])
print(score)

results['Fill zeros'] = score

184789.62179360405


# Wypełnienie pustych wartości wartością, która poprzedza wartość brakującą

In [12]:
train_df_cleaned = train_df.fillna(method='bfill', axis=0).fillna(0)  # wypełnia wartością poprzedzającą z danej kolumny, jeśli to niemożliwe, wstawia 0
test_df_cleaned = test_df.fillna(method='bfill', axis=0).fillna(0)  # wypełnia wartością poprzedzającą z danej kolumny, jeśli to niemożliwe, wstawia 0

cols_x = train_df_cleaned.select_dtypes(include=[np.number]).columns.difference(['Price'])  # wybiera tylko kolumny z wartosciami numerycznymi, za wyjątkiem kolumny z wartością referencyjną - wejście do klasyfikatora
cols_y = 'Price'  # - wyjście z klasyfikatora
score = score_dataset(train_df_cleaned[cols_x], test_df_cleaned[cols_x], train_df_cleaned[cols_y], test_df_cleaned[cols_y])
print(score)

results['Fill values before'] = score

/tmp/ipykernel_5527/3887095280.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  train_df_cleaned = train_df.fillna(method='bfill', axis=0).fillna(0)  # wypełnia wartością poprzedzającą z danej kolumny, jeśli to niemożliwe, wstawia 0
/tmp/ipykernel_5527/3887095280.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  test_df_cleaned = test_df.fillna(method='bfill', axis=0).fillna(0)  # wypełnia wartością poprzedzającą z danej kolumny, jeśli to niemożliwe, wstawia 0


191158.84383862824


# Podstawienie wartości średniej

In [13]:
from sklearn.impute import SimpleImputer

imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean') 
train_df_cleaned = train_df.select_dtypes(include=[np.number]).copy()
test_df_cleaned = test_df.select_dtypes(include=[np.number]).copy()#wybór tylko kolumn przechowujacych liczby, należy wykonać kopię obiektu

train_df_cleaned.loc[:] = imp_mean.fit_transform(train_df_cleaned)  # dopasowanie parametrów (średnich) i transformacja zbioru uczącego
test_df_cleaned[:]  = imp_mean.transform(test_df_cleaned)  # zastosowanie modelu do transformacji zbioru testowego (bez wyznaczania parametrów)

cols_x = train_df_cleaned.select_dtypes(include=[np.number]).columns.difference(['Price'])  # wybiera tylko kolumny z wartosciami numerycznymi, za wyjątkiem kolumny z wartością referencyjną - wejście do klasyfikatora
cols_y = 'Price'  # - wyjście z klasyfikatora
score = score_dataset(train_df_cleaned[cols_x], test_df_cleaned[cols_x], train_df_cleaned[cols_y], test_df_cleaned[cols_y])
print(score)

results['Fill mean'] = score

184695.51074689667


# Podstawienie wartości mediany

In [14]:
imp_mean = SimpleImputer(missing_values=np.nan, strategy='median') 
train_df_cleaned = train_df.select_dtypes(include=[np.number]).copy()
test_df_cleaned = test_df.select_dtypes(include=[np.number]).copy()#wybór tylko kolumn przechowujacych liczby, należy wykonać kopię obiektu

train_df_cleaned.loc[:] = imp_mean.fit_transform(train_df_cleaned)  # dopasowanie parametrów (średnich) i transformacja zbioru uczącego
test_df_cleaned[:]  = imp_mean.transform(test_df_cleaned)  # zastosowanie modelu do transformacji zbioru testowego (bez wyznaczania parametrów)

cols_x = train_df_cleaned.select_dtypes(include=[np.number]).columns.difference(['Price'])  # wybiera tylko kolumny z wartosciami numerycznymi, za wyjątkiem kolumny z wartością referencyjną - wejście do klasyfikatora
cols_y = 'Price'  # - wyjście z klasyfikatora
score = score_dataset(train_df_cleaned[cols_x], test_df_cleaned[cols_x], train_df_cleaned[cols_y], test_df_cleaned[cols_y])
print(score)

results['Fill median'] = score

185354.09575916964


# Podstawienie wartości mody

In [15]:
imp_mean = SimpleImputer(missing_values=np.nan, strategy='most_frequent') 
train_df_cleaned = train_df.select_dtypes(include=[np.number]).copy()
test_df_cleaned = test_df.select_dtypes(include=[np.number]).copy()#wybór tylko kolumn przechowujacych liczby, należy wykonać kopię obiektu

train_df_cleaned.loc[:] = imp_mean.fit_transform(train_df_cleaned)  # dopasowanie parametrów (średnich) i transformacja zbioru uczącego
test_df_cleaned[:]  = imp_mean.transform(test_df_cleaned)  # zastosowanie modelu do transformacji zbioru testowego (bez wyznaczania parametrów)

cols_x = train_df_cleaned.select_dtypes(include=[np.number]).columns.difference(['Price'])  # wybiera tylko kolumny z wartosciami numerycznymi, za wyjątkiem kolumny z wartością referencyjną - wejście do klasyfikatora
cols_y = 'Price'  # - wyjście z klasyfikatora
score = score_dataset(train_df_cleaned[cols_x], test_df_cleaned[cols_x], train_df_cleaned[cols_y], test_df_cleaned[cols_y])
print(score)

results['Fill frequency'] = score

184878.03623010026


In [25]:
for key in results:
    print(f'{key}: {results[key]}')
min_value = min(results)
print(f'Min values\n Type: {min_value}\n Value: {results[min_value]}')



Column erased: 183953.80955050973
Rows erased: 192095.01516671173
Fill zeros: 184789.62179360405
Fill values before: 191158.84383862824
Fill mean: 184695.51074689667
Fill median: 185354.09575916964
Fill frequency: 184878.03623010026
Min values
 Type: Column erased
 Value: 183953.80955050973
